In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rc('font',family='Times New Roman')

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import TensorDataset
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import math

In [3]:
# 创建一个模型配置类
class Config:
    def __init__(self, data_path='./Data/PRSA_Data/Data_Aotizhongxin.csv', timestep=1, train_size=0.75, model_name='model'):
        self.data_path = data_path
        self.timestep = timestep
        self.train_size = train_size
        self.model_name = model_name
        
    def modify_params(self, param, value):
        if param == 'data_path':
            self.data_path = value
        elif param == 'timestep':
            self.timestep = value
        elif param == 'train_size':
            self.train_size = value
        elif param == 'model_name':
            self.model_name = value
        else:
            raise SystemExit('The param is out the range OR value type is false')

In [4]:
# 加载数据并标准化（归一化）
# 数据标准化（归一化）
# 标准化：对原始数据进行变换把数据变换到均值为0,方差为1范围内
# 归一化：对每个特征缩放到给定范围(默认[0,1])
# 一般只对自变量X进行标准化（归一化）
def read_data(data_path):
    df_data = pd.read_csv(data_path, index_col=0)
    # 采用IQR异常值检验和线性插值
    df = df_data.copy()
    df = IQR(df_data, df)
    df = df.interpolate()
    scaler = StandardScaler()
    data = pd.DataFrame(scaler.fit_transform(np.array(df)))
    df.reset_index(inplace=True)
    data = pd.concat([data.iloc[:,1:], df.iloc[:,1]], axis=1,join='outer')
    data = np.array(data)
    return data

def IQR(data, new_data):
    for i in range(len(np.array(data.columns))):
        df_25 = data.iloc[:,i].quantile(0.25)
        df_75 = data.iloc[:,i].quantile(0.75)
        IQR = df_75 - df_25
        lower_limit = df_25 - 1.5*IQR
        upper_limit = df_75 + 1.5*IQR
        new_data.iloc[:,i] = data.iloc[:,i][(data.iloc[:,i]>lower_limit) & (data.iloc[:,i]<upper_limit)]
        return new_data

In [5]:
# 划分训练集、测试集
def split_data(data, timestep, train_size):
    data_X = []
    data_Y = []
    
    # 将整个窗口的数据保存到X中，将未来timestep保存到Y中
    for index in range(len(data)- timestep):
        data_X.append(data[index: index + timestep])
        data_Y.append(data[index + timestep][-1])
    
    dataX = np.array(data_X)
    dataY = np.array(data_Y)
    
    train_size = int(np.round(train_size * dataX.shape[0]))
    
    x_train = dataX[: train_size, :].reshape(-1, timestep, data.shape[1])
    y_train = dataY[: train_size].reshape(-1, 1)
    
    x_test = dataX[train_size:, :].reshape(-1, timestep, data.shape[1])
    y_test = dataY[train_size:].reshape(-1, 1)
    
    x_train = x_train.reshape(x_train.shape[0], (x_train.shape[1] * x_train.shape[2]))
    x_test = x_test.reshape(x_test.shape[0], (x_test.shape[1] * x_test.shape[2]))
    
    x_train = pd.DataFrame(x_train)
    x_test = pd.DataFrame(x_test)
        
    return [x_train, y_train, x_test, y_test]

In [6]:
def split_data_cnn(data, timestep, train_size):
    data_X = []
    data_Y = []
    
    for index in range(len(data) - timestep):
        data_X.append(data[index: index + timestep])
        data_Y.append(data[index + timestep][-1])
        
    dataX = np.array(data_X)
    dataY = np.array(data_Y)
    
    train_size = int(np.round(train_size * dataX.shape[0]))
    x_train = dataX[: train_size, :].reshape(-1, timestep, data.shape[1])
    y_train = dataY[: train_size]
    
    x_test = dataX[train_size:, :].reshape(-1, timestep, data.shape[1])
    y_test = dataY[train_size: ]
    
    return [x_train, y_train, x_test, y_test]

In [7]:
# 将数据转为tensor，并加载为迭代器
def tensor_load_cnn(x_train, y_train, x_test, y_test, batch_size):
    x_train_tensor = torch.from_numpy(x_train).to(torch.float32)
    y_train_tensor = torch.from_numpy(y_train).to(torch.float32)
    x_test_tensor = torch.from_numpy(x_test).to(torch.float32)
    y_test_tensor = torch.from_numpy(y_test).to(torch.float32)
    
    train_data = TensorDataset(x_train_tensor, y_train_tensor)
    test_data = TensorDataset(x_test_tensor, y_test_tensor)
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size, False)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size, False)
    return x_test_tensor, y_test_tensor, train_loader, test_loader

In [8]:
# 将数据转为tensor，并加载为迭代器
def tensor_load(x_train, y_train, x_test, y_test, batch_size):
    x_train_tensor = torch.from_numpy(x_train.values).to(torch.float32)
    y_train_tensor = torch.from_numpy(y_train).to(torch.float32)
    x_test_tensor = torch.from_numpy(x_test.values).to(torch.float32)
    y_test_tensor = torch.from_numpy(y_test).to(torch.float32)
    
    train_data = TensorDataset(x_train_tensor, y_train_tensor)
    test_data = TensorDataset(x_test_tensor, y_test_tensor)
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size, False)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size, False)
    return x_test_tensor, y_test_tensor, train_loader, test_loader

In [9]:
# 利用各种机器学习算法进行模型训练

In [10]:
# 利用训练并保存好的模型进行预测
def prediction(x_test_tensor, y_test_tensor, config):
    save_path = './Model/{}.pth'.format(config.model_name)
    model = torch.load(save_path)
    device = torch.device("cpu")
    model = model.to(device)
    y_pre = model(x_test_tensor)
    y_pre = y_pre.to(torch.device('cpu')).detach().numpy()
    NSE =r2_score(y_test_tensor,y_pre)
    MAE =mean_absolute_error(y_test_tensor,y_pre)
    return y_pre, NSE, MAE

In [11]:
# 保存预测结果
def save_results(y_test_tensor, y_pre, config):
    file_path = './Results/{}.csv'.format(config.model_name)
    df = pd.read_csv(config.data_path, index_col=0)
    train_size = int(np.round(config.train_size * df.shape[0]))
    index = df.index.values[train_size:]
    index = pd.to_datetime(pd.DataFrame(index).iloc[:,0]).values
    y_test_tensor = torch.squeeze(y_test_tensor).tolist()
    y_pre = y_pre.ravel()
    result = pd.DataFrame({'datetime': index, 'Measured values': y_test_tensor, 'Predicted values':y_pre})
    result.to_csv(file_path, index=False, sep=',')
    
def save_results_cnn(y_test_tensor, y_pre, config):
    file_path = './Results/{}.csv'.format(config.model_name)
    y_test_tensor = torch.squeeze(y_test_tensor).tolist()
    y_pre = y_pre.ravel()
    result = pd.DataFrame({'Measured values': y_test_tensor, 'Predicted values':y_pre})
    result.to_csv(file_path, index=False, sep=',')

#### 1. MLPNN

In [10]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=8, feature_size=11, hidden_size1=100, hidden_size2=50, output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.hidden_size1 = hidden_size1
        self.output_size = output_size
        self.hidden_size2 = hidden_size2
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'hidden_size1':
            self.hidden_size1 = value
        elif param == 'hidden_size_2':
            self.hidden_size_2 = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')

# 定义MLP网络
class MLP(nn.Module):
    def __init__(self, feature_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(in_features=feature_size, out_features=hidden_size1, bias=True)
        self.hidden2 = nn.Linear(hidden_size1, hidden_size1)
        self.hidden3 = nn.Linear(hidden_size1, hidden_size2)
        self.predict = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x.to(device)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.predict(x)
        return x[:, 0]
    
# 模型训练并保存
def MLPNN(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义MLP网络
    model = MLP(nn_config.feature_size, nn_config.hidden_size1, nn_config.hidden_size2, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.SGD(model.parameters(), lr=nn_config.learning_rate)
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train)
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
        train_loss_plot.append(min(loss_plot))
            
    # 模型验证
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred, y_test.reshape(-1, 1))
            test_loss_plot.append(test_loss.item())
    
    save_path = './Model/{}.pth'.format(config.model_name)        
    if test_loss < nn_config.best_loss:
        nn_config.modify_params('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot

In [30]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.初始化模型超参数
nn_config = NN_config()
# 3.根据使用模型修改名称
config.modify_params('model_name', 'MLP')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = MLPNN(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor, y_test_tensor, config)
# 9.结果保存
save_results(y_test_tensor, y_pre, config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 877/877 [00:00<00:00, 2560.72it/s]

NSE= 0.9411589341083014
MAE= 9.7638035


#### 2.循环神经网络（RNN）

#### 2.1 传统RNN

In [11]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=8, feature_size=11, hidden_size=128, num_layers=2, output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'hidden_size':
            self.hidden_size = value
        elif param == 'num_layers':
            self.num_layers = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')
        
# 定义RNN网络
# 训练过程中可能出现预测结果为一条直线的情况，解决办法请参照：https://blog.csdn.net/m0_47256162/article/details/128720691#:~:text=%E5%AF%B9%E4%BA%8E%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97%E6%95%B0%E6%8D%AE,%E5%B8%B8%E8%A7%81%E7%9A%84%E5%BD%B1%E5%93%8D%E5%9B%A0%E7%B4%A0%E3%80%82
class RNN(nn.Module):
    def __init__(self, feature_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # feature_size为特征维度，就是每个时间点对应的特征数量
        self.rnn = nn.RNN(feature_size, hidden_size, num_layers, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden=None):
        batch_size = x.shape[0]  # 获取批次大小
        if hidden is None:
            h_0 = x.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
        else:
            h_0 = hidden
        x = x.view(len(x), 1, -1)
        output, h_0 = self.rnn(x, h_0)
        output = self.fc(output)  # 形状为batch_size * timestep, 1
        return output[:, -1, :]  # 只需要返回最后一个时间切片的数据
    
# 模型训练并保存
def RNN_model(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义RNN网络
    model = RNN(nn_config.feature_size, nn_config.hidden_size, nn_config.num_layers, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=nn_config.learning_rate)
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train)
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
        train_loss_plot.append(min(loss_plot))
            
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred, y_test.reshape(-1, 1))
            test_loss_plot.append(test_loss)
    
    save_path = './Model/{}.pth'.format(config.model_name)
    if test_loss < nn_config.best_loss:
        nn_config.modify('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot

In [28]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.初始化模型超参数
nn_config = NN_config(learning_rate=1e-6, epochs=60)
# 3.根据使用模型修改名称
config.modify_params('model_name', 'rnn')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = RNN_model(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor, y_test_tensor, config)
# 9.结果保存
save_results(y_test_tensor, y_pre, config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 877/877 [00:00<00:00, 2636.96it/s]

NSE= 0.9449101191576368
MAE= 8.836823


#### 2.2 LSTM

In [31]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=16, feature_size=11, hidden_size=128, num_layers=2, output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'hidden_size':
            self.hidden_size = value
        elif param == 'num_layers':
            self.num_layers = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')
        
# 定义LSTM网络
# 若使用BiLSTM，需设置nn.LSTM中的超参数bidirectional=True
class LSTM(nn.Module):
    def __init__(self, feature_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # feature_size为特征维度，就是每个时间点对应的特征数量
        self.lstm = nn.LSTM(feature_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.7)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden=None):
        batch_size = x.shape[0]  # 获取批次大小
        if hidden is None:
            h_0 = x.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
            c_0 = x.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
        else:
            h_0, c_0 = hidden
        x = x.view(len(x), 1, -1)
        output, (h_0, c_0) = self.lstm(x, (h_0, c_0))
        output = self.dropout(output)
        output = F.sigmoid(self.fc(output))  # 形状为batch_size * timestep, 1
        return output[:, -1, :]  # 只需要返回最后一个时间切片的数据
    
# 模型训练并保存
def LSTM_model(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义RNN网络
    model = LSTM(nn_config.feature_size, nn_config.hidden_size, nn_config.num_layers, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=nn_config.learning_rate)
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train)
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
        train_loss_plot.append(min(loss_plot))
            
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred, y_test.reshape(-1, 1))
            test_loss_plot.append(test_loss)
    
    save_path = './Model/{}.pth'.format(config.model_name)
    if test_loss < nn_config.best_loss:
        nn_config.modify('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot

In [33]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.初始化模型超参数
nn_config = NN_config(learning_rate=1e-3, epochs=60)
# 3.根据使用模型修改名称
config.modify_params('model_name', 'lstm')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = LSTM_model(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor, y_test_tensor, config)
# 9.结果保存
save_results(y_test_tensor, y_pre, config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 439/439 [00:00<00:00, 2347.61it/s]

NSE= -1.2348134381480818
MAE= 72.79566


#### 2.3 GRU

In [71]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=8, feature_size=11, hidden_size=128, num_layers=2, output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'hidden_size':
            self.hidden_size = value
        elif param == 'num_layers':
            self.num_layers = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')
        
# 定义GRU网络
class GRU(nn.Module):
    def __init__(self, feature_size, hidden_size, num_layers, output_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # feature_size为特征维度，就是每个时间点对应的特征数量
        self.gru = nn.GRU(feature_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden=None):
        batch_size = x.shape[0]  # 获取批次大小
        if hidden is None:
            h_0 = x.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
        else:
            h_0 = hidden
        x = x.view(len(x), 1, -1)
        output, h_0 = self.gru(x, h_0)
        output = self.fc(output)  # 形状为batch_size * timestep, 1
        return output[:, -1, :]  # 只需要返回最后一个时间切片的数据
    
# 模型训练并保存
def GRU_model(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义RNN网络
    model = GRU(nn_config.feature_size, nn_config.hidden_size, nn_config.num_layers, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=nn_config.learning_rate)
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train)
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
        train_loss_plot.append(min(loss_plot))
            
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred, y_test.reshape(-1, 1))
            test_loss_plot.append(test_loss)
    
    save_path = './Model/{}.pth'.format(config.model_name)
    if test_loss < nn_config.best_loss:
        nn_config.modify('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot

In [72]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.初始化模型超参数
nn_config = NN_config(batch_size=16, learning_rate=1e-3, epochs=60)
# 3.根据使用模型修改名称
config.modify_params('model_name', 'gru')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = GRU_model(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor, y_test_tensor, config)
# 9.结果保存
save_results(y_test_tensor, y_pre, config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 439/439 [00:00<00:00, 2420.10it/s]


NSE= 0.9391883507143608
MAE= 9.837735


#### 3.卷积神经网络（CNN）

##### 3.1 传统CNN

In [75]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=16, feature_size=11, out_channels=[10, 10, 10], output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.output_size = output_size
        self.out_channels = out_channels
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'out_channels':
            self.out_channels = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')
    
# 定义CNN网络
class CNN(nn.Module):
    def __init__(self, feature_size, out_channels, output_size, kernel_size=3, stride=2, padding=0):
        super(CNN, self).__init__()
        self.conv1d_1 = nn.Conv1d(feature_size, out_channels[0], kernel_size=kernel_size, stride=stride, padding=padding)
        self.conv1d_2 = nn.Conv1d(out_channels[0], out_channels[1], kernel_size=kernel_size, stride=stride, padding=padding)
        self.conv1d_3 = nn.Conv1d(out_channels[1], out_channels[2], kernel_size=kernel_size, stride=stride, padding=padding)
        self.maxpool1 = nn.AdaptiveMaxPool1d(output_size=20)
        self.maxpool2 = nn.AdaptiveMaxPool1d(output_size=15)
        self.maxpool3 = nn.AdaptiveMaxPool1d(output_size=10)
        self.fc = nn.Linear(out_channels[2] * 10, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1d_1(x)
        x = self.maxpool1(x)
        x = self.conv1d_2(x)
        x = self.maxpool2(x)
        x = self.conv1d_3(x)
        x = self.maxpool3(x)
        x = x.reshape(-1, x.shape[1] * x.shape[2])
        x = self.fc(x)
        return x

# 模型训练并保存
def CNN_model(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义CNN网络
    model = CNN(nn_config.feature_size, nn_config.out_channels, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=nn_config.learning_rate)        
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train.transpose(1, 2))
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
        train_loss_plot.append(min(loss_plot))
    # 模型验证
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test.transpose(1, 2))
            test_loss = loss_function(y_test_pred, y_test.reshape(-1, 1))
            test_loss_plot.append(test_loss)
                
    save_path = './Model/{}.pth'.format(config.model_name)
    if test_loss < nn_config.best_loss:
        nn_config.modify('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot

In [125]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8, timestep=12)
# 2.初始化模型超参数
nn_config = NN_config(batch_size=8, learning_rate=1e-6, epochs=150)
# 3.根据使用模型修改名称
config.modify_params('model_name', 'cnn')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data_cnn(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load_cnn(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = CNN_model(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor.transpose(1, 2), y_test_tensor, config)
# 9.结果保存
save_results_cnn(y_test_tensor.reshape(y_test_tensor.shape[0], 1), y_pre.reshape(-1), config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 877/877 [00:00<00:00, 2737.50it/s]

NSE= 0.8185555120875392
MAE= 17.525034


##### 3.2 TCN

In [20]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=16, feature_size=11, num_channels=[32, 64, 128, 256], output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.output_size = output_size
        self.num_channels = num_channels
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'num_channels':
            self.num_channels = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')

# 定义TCN网络
class TCN(nn.Module):
    def __init__(self, feature_size, num_channels, output_size, kernel_size=3, dropout=0.2):
        super(TCN, self).__init__()
        self.feature_size = feature_size
        self.output_size = output_size
        self.num_channels = num_channels
        self.kernel_size = kernel_size
        self.relu = nn.ReLU()        
        # 定义卷积层和dropout层
        self.model_list = nn.ModuleList([nn.Conv1d(feature_size, num_channels[0], kernel_size), 
                                         nn.Conv1d(num_channels[0], num_channels[1], kernel_size),
                                         nn.Conv1d(num_channels[1], num_channels[2], kernel_size),
                                         nn.Conv1d(num_channels[2], num_channels[3], kernel_size)])
        self.dropout = nn.Dropout(dropout)
        # 定义全连接层
        self.fc = nn.Linear(num_channels[-1], output_size)
        
    def forward(self, x):
        # 将数据维度从（batch_size, seq_len, input_size）变成（batch_size, input_size, seq_len）
        x = x.permute(0, 2, 1)
        # 通过卷积层和dropout进行特征提取
        for layer in self.model_list:
            x = layer(x)
            x = self.relu(x)
            x = self.dropout(x)
        # 将卷积层的输出求平均并输入全连接层得到最终输出
        x = x.mean(dim=2)
        x = self.fc(x)
        return x
    
# 模型训练并保存
def TCN_model(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义TCN网络
    model = TCN(nn_config.feature_size, nn_config.num_channels, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=nn_config.learning_rate)
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train)
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
        train_loss_plot.append(min(loss_plot))
    # 模型验证
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred, y_test.reshape(-1 ,1))
            test_loss_plot.append(test_loss)
            
    save_path = './Model/{}.pth'.format(config.model_name)
    if test_loss < nn_config.best_loss:
        nn_config.modify('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot 

In [22]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8, timestep=12)
# 2.初始化模型超参数
nn_config = NN_config(batch_size=8, learning_rate=1e-7, epochs=50)
# 3.根据使用模型修改名称
config.modify_params('model_name', 'TCN')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data_cnn(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load_cnn(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = TCN_model(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor, y_test_tensor, config)
# 9.结果保存
save_results_cnn(y_test_tensor, y_pre, config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 877/877 [00:00<00:00, 2088.15it/s]

NSE= 0.6220382283283994
MAE= 27.732985


#### 4.注意力机制（Attention）

##### 4.1 Transformer

In [26]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=16, feature_size=11, hidden_size=256, num_layers=2, output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'hidden_size':
            self.hidden_size = value
        elif param == 'num_layers':
            self.num_layers = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')

# 定义位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # 初始化Shape为(max_len, d_model)的positional encoding(pe)
        pe = torch.zeros(max_len, d_model)
        # 初始化一个tensor[[0, 1, 2, 3, ...]]
        position = torch.arange(0, max_len).unsqueeze(1)
        # 这里是sin和cos括号中的内容，通过e和ln进行交换
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0)/d_model))
        # 计算PE(pos, 2i)
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算PE(pos, 2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)
        
    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)
    
# 定义Transformer网络
class Transformer(nn.Module):
    def __init__(self, hidden_size, num_layers, feature_size, output_size, feedforward_dim=32, num_head=1, transformer_num_layers=1, dropout=0.3, max_len=1):
        super(Transformer, self).__init__()
        self.hidden_size = hidden_size # 隐层大小
        self.num_layers = num_layers # lstm层数
        # feature_size为特征维度，就是每个时间点对应的特征数量
        self.lstm = nn.LSTM(feature_size, hidden_size, num_layers, batch_first=True)
        # 位置编码层
        self.position_encoding = PositionalEncoding(hidden_size, dropout, max_len)
        # 编码层
        self.encoder_layer = nn.TransformerEncoderLayer(hidden_size, num_head, feedforward_dim, dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, transformer_num_layers)
        # 输出层
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, output_size)
        # 激活函数
        self.relu = nn.ReLU()
        
    def forward(self, x, hidden=None):
        batch_size = x.shape[0]
        # 初始化隐层状态
        if hidden is None:
            h_0 = x.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
            c_0 = x.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
        else:
            h_0, c_0 = hidden
        # LSTM运算    
        output, (h_0, c_0) = self.lstm(x, (h_0, c_0))
        # 维度为【序列长度，批次，嵌入向量维度】
        output = self.position_encoding(output)
        output = self.transformer(output)
        # 将每个数据的输出向量取均值，也可以随意去一个标记输出结果，维度为【批次，嵌入向量维度】
        output = output.mean(axis=1)
        output = self.fc1(output)
        output = self.relu(output)
        output = self.fc2(output)
        return output

# 模型训练并保存
def Transformer_model(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义Transformer网络
    model = Transformer(nn_config.hidden_size, nn_config.num_layers, nn_config.feature_size, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=nn_config.learning_rate)
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train)
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
        train_loss_plot.append(min(loss_plot))
    # 模型验证
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred, y_test.reshape(-1, 1))
            test_loss_plot.append(test_loss)
    
    save_path = './Model/{}.pth'.format(config.model_name)
    if test_loss < nn_config.best_loss:
        nn_config.modify('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot

In [28]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8, timestep=12)
# 2.初始化模型超参数
nn_config = NN_config(batch_size=16, learning_rate=1e-6, epochs=100)
# 3.根据使用模型修改名称
config.modify_params('model_name', 'Transformer')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data_cnn(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load_cnn(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = Transformer_model(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor, y_test_tensor, config)
# 9.结果保存
save_results_cnn(y_test_tensor, y_pre, config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 439/439 [00:00<00:00, 889.84it/s]


NSE= 0.9236650965872573
MAE= 11.5833845


##### 4.2 Informer

In [36]:
# 创建网络超参数类
class NN_config:
    def __init__(self, batch_size=16, feature_size=11, output_size=1, epochs=50, learning_rate=1e-8, best_loss=0):
        self.batch_size = batch_size
        self.feature_size = feature_size
        self.output_size = output_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.best_loss = best_loss
        
    def modify_params(self, param, value):
        if param == 'batch_size':
            self.batch_size = value
        elif param == 'feature_size':
            self.feature_size = value
        elif param == 'output_size':
            self.output_size = value
        elif param == 'epochs':
            self.epochs = value
        elif param == 'learning_rate':
            self.learning_rate = value
        elif param == 'best_loss':
            self.best_loss = value
        else:
            raise SystemExit('The param is out the range OR value type is false')
        
# 定义Informer网络
class Informer(nn.Module):
    def __init__(self, feature_size, output_size, num_encoder_layers=2, num_decoder_layers=1, d_model=11, nhead=1, dim_feedforward=32, dropout=0.2, activation='relu'):
        super(Informer, self).__init__()
        self.feature_size = feature_size
        self.output_size = output_size
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.d_model = d_model
        self.nhead = nhead
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.activation = activation
        # 编码器
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, activation=activation)
        self.encoder = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_encoder_layers)
        # 解码器
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, activation=activation)
        self.decoder = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=num_decoder_layers)
        # 全连接层
        self.fc = nn.Linear(d_model, output_size)
    
    def forward(self, x):
        x = x.permute(1, 0, 2)
        # 编码器处理输入序列
        enc_output = self.encoder(x)
        # 解码器将编码器输出作为输入，并预测目标序列
        dec_output = self.decoder(enc_output, enc_output)
        # 取最后一个时间步长的输出，并通过全连接层得到最终输出
        output = self.fc(dec_output[-1])
        return output

# 模型训练并保存
def Informer_model(train_loader, test_loader, config, nn_config):
    # 选择训练硬件设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义Insformer网络
    model = Informer(nn_config.feature_size, nn_config.output_size).to(device)
    # 定义损失函数
    loss_function = nn.MSELoss().to(device)
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=nn_config.learning_rate)
    # 模型训练
    train_loss_plot = []
    test_loss_plot = []
    for epoch in range(nn_config.epochs):
        model.train()
        running_loss = 0
        loss_plot = []
        train_bar = tqdm(train_loader)
        for data in train_bar:
            x_train, y_train = data
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            optimizer.zero_grad()
            y_train_pred = model(x_train)
            loss = loss_function(y_train_pred, y_train.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            loss_plot.append(loss.item())
            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, nn_config.epochs, loss)
    # 模型验证
    model.eval()
    test_loss = 0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred, y_test.reshape(-1, 1))
            test_loss_plot.append(test_loss)
            
    save_path = './Model/{}.pth'.format(config.model_name)
    if test_loss < nn_config.best_loss:
        nn_config.modify('best_loss', test_loss)
        torch.save(model, save_path)
    else:
        torch.save(model, save_path)
    return train_loss_plot, test_loss_plot    

In [38]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8, timestep=12)
# 2.初始化模型超参数
nn_config = NN_config(batch_size=16, learning_rate=1e-5, epochs=100)
# 3.根据使用模型修改名称
config.modify_params('model_name', 'Informer')
# 4.读取数据
data = read_data(config.data_path)
# 5.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data_cnn(data, config.timestep, config.train_size)
# 6.将数据格式转为tensor
x_test_tensor, y_test_tensor, train_loader, test_loader = tensor_load_cnn(x_train, y_train, x_test, y_test, nn_config.batch_size)
# 7.模型训练
train_loss_plot, test_loss_plot = Informer_model(train_loader, test_loader, config, nn_config)
# 8.模型预测
y_pre, NSE, MAE = prediction(x_test_tensor, y_test_tensor, config)
# 9.结果保存
save_results_cnn(y_test_tensor, y_pre, config)
# 10.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

100%|██████████| 439/439 [00:00<00:00, 631.67it/s]

NSE= 0.032096411021306004
MAE= 38.674965
